In [ ]:
# Install required packages
!pip install -q transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
parsivar 0.2.3 requires nltk==3.4.5, but you have nltk 3.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached nltk-3.4.5-py3-none-any.whl
  Attempting uninstall: nltk
    Found existing installation: nltk 3.3
    Uninstalling nltk-3.3:
      Successfully uninstalled nltk-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hazm 0.7.0 requires nltk==3.3, but you have nltk 3.4.5 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in i

In [ ]:
# Import required packages
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from tqdm.notebook import tqdm

from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf

from keras.layers import GlobalMaxPooling1D
from keras.layers import *
from keras.models import Model

from transformers import TFBertModel, TFBertForSequenceClassification, BertModel
from transformers.modeling_tf_utils import get_initializer

from sklearn.model_selection import train_test_split
import sklearn.metrics as skm

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


### Load the data using Pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
emotion_dataset_train =  pd.read_excel('drive/MyDrive/ShortPersianEmo 2023 Code/Data/ArmanEmo/train_fa.xlsx', header=None).dropna()
emotion_dataset_test = pd.read_excel('drive/MyDrive/ShortPersianEmo 2023 Code/Data/ArmanEmo/test_fa.xlsx', header=None).dropna()

x_train = emotion_dataset_train[0]
y_train = emotion_dataset_train[1]

x_test = emotion_dataset_test[0]
y_test = emotion_dataset_test[1]

In [ ]:
from collections import Counter
cnt = Counter(y_train)
cnt = dict(cnt)
print(cnt)

{'SAD': 902, 'HATE': 515, 'OTHER': 1685, 'FEAR': 759, 'ANGRY': 926, 'HAPPY': 625, 'SURPRISE': 740}


### Normalization / Preprocessing

In [ ]:
data_train = emotion_dataset_train[[0, 1]]
data_train.columns = ['comment', 'label']

data_test = emotion_dataset_test[[0, 1]]
data_test.columns = ['comment', 'label']

## Train,Validation,Test split

In [ ]:
test = data_test
train = data_train

x_train, y_train = train['comment'].values.tolist(), train['label'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['label'].values.tolist()

print(train.shape)
print(test.shape)

(6152, 2)
(1151, 2)


## TensorFlow

In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-base-parsbert-uncased'
OUTPUT_PATH = '/content/Arman_DS/tensorflow_model.bin'

# os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
labels = list(sorted(train['label'].unique()))
train['label_id'] = train['label'].apply(lambda t: labels.index(t))
test['label_id'] = test['label'].apply(lambda t: labels.index(t))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(MODEL_NAME_OR_PATH, **{ 'label2id': label2id, 'id2label': id2label})

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/config.json
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-base-parsbert-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_lay

### Input Embeddings / Dataset

In [ ]:
class InputExample:
    """ A single example for simple sequence classification. """
    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label
def make_examples(tokenizer, x, y=None, maxlen=MAX_LEN, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)
    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)
        if isinstance(_x, str):
            text_a = _x
            text_b = None
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    features = glue_convert_examples_to_features(
        examples,
        tokenizer,
        maxlen,
        output_mode=output_mode,
        label_list=list(np.unique(y)))
    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []
    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)
    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels
    return [xdata, ydata], features

In [ ]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, np.asarray(train['label_id']), maxlen=MAX_LEN)

test_dataset_base, test_examples = make_examples(tokenizer, x_test, np.asarray(test['label_id']), maxlen=MAX_LEN)
[xtest, ytest], test_examples = make_examples(tokenizer, x_test, np.asarray(test['label_id']), maxlen=MAX_LEN, is_tf_dataset=False)

  0%|          | 0/6152 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
*** Example ***
guid: 0
features: InputFeatures(input_ids=[2, 3062, 46923, 25594, 331, 9598, 12571, 55226, 72912, 21405, 2297, 21773, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0

  0%|          | 0/6152 [00:00<?, ?it/s]

  0%|          | 0/1151 [00:00<?, ?it/s]

*** Example ***
guid: 0
features: InputFeatures(input_ids=[2, 7764, 1159, 13413, 1160, 2389, 10308, 24198, 1206, 33930, 44112, 2045, 1, 333, 2038, 4487, 15, 2699, 31613, 25392, 2045, 302, 2318, 2036, 2188, 7880, 300, 2318, 2036, 7594, 300, 2318, 2036, 2128, 24277, 1160, 15, 3452, 50033, 2462, 15, 15, 1, 12833, 1, 9508, 3806, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 

  0%|          | 0/1151 [00:00<?, ?it/s]

  0%|          | 0/1151 [00:00<?, ?it/s]

*** Example ***
guid: 0
features: InputFeatures(input_ids=[2, 7764, 1159, 13413, 1160, 2389, 10308, 24198, 1206, 33930, 44112, 2045, 1, 333, 2038, 4487, 15, 2699, 31613, 25392, 2045, 302, 2318, 2036, 2188, 7880, 300, 2318, 2036, 7594, 300, 2318, 2036, 2128, 24277, 1160, 15, 3452, 50033, 2462, 15, 15, 1, 12833, 1, 9508, 3806, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 

  0%|          | 0/1151 [00:00<?, ?it/s]

In [ ]:
def get_training_dataset(dataset, batch_size):
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    return dataset

def get_validation_dataset(dataset, batch_size):
    dataset = dataset.batch(batch_size)
    return dataset

In [ ]:
train_dataset = get_training_dataset(train_dataset_base, TRAIN_BATCH_SIZE)
test_dataset = get_validation_dataset(test_dataset_base, VALID_BATCH_SIZE)

train_steps = len(train_examples) // TRAIN_BATCH_SIZE
test_steps = len(test_examples) // VALID_BATCH_SIZE

### Model

In [ ]:
import math
def ceiltoup(x):
  return math.ceil(x * 100) / 100.0

f1_macro = []
accuracy = []
for i in range(10):
  input_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
  attention_masks = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
  token_type_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="token_type_ids")
  model = TFBertModel.from_pretrained(MODEL_NAME_OR_PATH, config=config, from_pt=True)
  model.trainable = True
  output = model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
  output = Dense(len(labels), activation='softmax', name='dense_out')(output['pooler_output'])
  model = Model(inputs=[input_ids, attention_masks, token_type_ids], outputs=output)
  optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
  EarlyStopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=1e-3, patience=2, verbose=0, mode='auto', restore_best_weights=True)
  r = model.fit( train_dataset, validation_data=test_dataset, steps_per_epoch=train_steps, validation_steps=test_steps, epochs=20, batch_size=32, verbose=0, callbacks=[EarlyStopping_callback])
  predictions = model.predict(xtest)
  ypred = predictions.argmax(axis=-1).tolist()
  f1_macro.append(skm.f1_score(ytest, ypred, average="macro"))
  accuracy.append(skm.accuracy_score(ytest, ypred, normalize=True))
print('f1 macro : ' + str(f1_macro) + '\nmean f1 macro : ' + str(ceiltoup(np.mean(f1_macro))))
print('accuracy : ' + str(accuracy) + '\nmean accuracy : ' + str(ceiltoup(np.mean(accuracy))))

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
PyTorch checkpoint contains 240,435,010 parameters
Loaded 162,841,344 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or 

36/36 [==============================] - 13s 283ms/step


loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
PyTorch checkpoint contains 240,435,010 parameters
Loaded 162,841,344 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or 

36/36 [==============================] - 13s 290ms/step


loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
PyTorch checkpoint contains 240,435,010 parameters
Loaded 162,841,344 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or 

...

36/36 [==============================] - 13s 291ms/step


loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
PyTorch checkpoint contains 240,435,010 parameters
Loaded 162,841,344 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or 

36/36 [==============================] - 13s 290ms/step


loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/pytorch_model.bin
PyTorch checkpoint contains 240,435,010 parameters
Loaded 162,841,344 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or 

36/36 [==============================] - 13s 290ms/step
f1 macro : [0.65, 0.64, 0.66, 0.65, 0.67, 0.67, 0.63, 0.63, 0.64, 0.65]
mean f1 macro : 0.65
accuracy : [0.66, 0.65, 0.67, 0.66, 0.69, 0.69, 0.64, 0.66, 0.66, 0.68]
mean accuracy : 0.67
